In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
import tqdm
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import datetime

# Requests
import requests

from time import sleep
import random

# Para trabajar con archivos dotenv y los tokens
# -----------------------------------------------------------------------
import os
import dotenv
dotenv.load_dotenv(dotenv_path="../src/.env")
# Para Importar desde rutas recursivas
# -----------------------------------------------------------------------
import sys
sys.path.append("../") #Para poder importar
from src.SupportVuelos import get_airport

# Aeropuertos
- MAD: 95565077
- OSL:
- NCE:

In [5]:
aeropuertos = ["madrid","niza","oslo"]
resultados = [get_airport(i) for i in aeropuertos]

In [6]:
resultados
df = pd.DataFrame()
for i in range(0,len(resultados)):
    df_temp = pd.DataFrame({
        "skyId": resultados[i][0],
        "entityId": resultados[i][1],
        "flightPlaceType" : resultados[i][2],
        "localizedName" : resultados[i][3]}))

[['MAD', '95565077', 'AIRPORT', 'Madrid-Barajas'],
 ['NCE', '95673860', 'AIRPORT', 'Niza'],
 ['OSL', '128667763', 'AIRPORT', 'Oslo-Gardermoen']]